# Camera Image Capture

## Setup python dependencies

```bash
# to install picamera2
$ sudo apt update
$ sudo apt upgrade
$ sudo apt install -y python3-picamera2

# create venv using system packages
$ python -m venv --system-site-packages .venv

# activate venv and install additional dependencies
$ source .venv/bin/activate
$ pip install torch torchvision torchaudio opencv-python matplotlib jupyter
```

In [1]:
import cv2
import IPython.display as ipython
import ipywidgets as widgets


In [2]:
import threading
from time import sleep
from typing import Tuple
import warnings

try:
    from libcamera import controls
    from picamera2 import Picamera2
    is_libcamera = True
except ModuleNotFoundError:
    warnings.warn("libcamera module not found. Use openCV instead.")
    is_libcamera = False


class VideoCapture(threading.Thread):
    def __init__(self, cam, resolution):
        super().__init__()
        self.resolution = resolution
        self.cam = cam
        self.frame = None
        self.capture = False
    
    def run(self):
        try:
            while self.capture:
                if is_libcamera:
                    self.frame = self.cam.capture_array()
                else:
                    _, self.frame = self.cam.read()
        finally:
            pass

    def start(self):
        self.capture = True
        if is_libcamera:
            self.cam = Picamera2()
            # Picamera2.create_preview_configuration will generate a configuration suitable for displaying camera preview images on the display, or prior to capturing a still image
            # Picamera2.create_still_configuration will generate a configuration suitable for capturing a high-resolution still image
            # Picamera2.create_video_configuration will generate a configuration suitable for recording video files
            self.cam.configure(self.cam.create_preview_configuration(main={"format": 'RGB888', "size": self.resolution}))
            # There are three autoFocus modes: Manual, Auto, and Continuous
            self.cam.set_controls({"AfMode": controls.AfModeEnum.Continuous})
            self.cam.start()
        else:
            pass
        sleep(1)  # delay for starting the cam
        super().start()
    
    def stop(self):
        self.capture = False
        self.join()
        if is_libcamera:
            self.cam.stop()
        else:
            self.cam.release()


class Camera():
    def __init__(self, resolution: Tuple = (320, 320)):
        self.resolution = resolution
        self.video_capture = None  # video capture thread

    def start(self):
        self.stop()
        
        self.cam = cv2.VideoCapture(0)
        self.cam.set(cv2.CAP_PROP_FRAME_WIDTH, self.resolution[0])
        self.cam.set(cv2.CAP_PROP_FRAME_HEIGHT, self.resolution[1])    
        
        self.video_capture = VideoCapture(self.cam, self.resolution)
        self.video_capture.start()
        

    def stop(self):
        if self.video_capture is not None:
            self.video_capture.stop()
            self.video_capture = None

    def get_frame(self):
        if self.video_capture is not None:
            return self.video_capture.frame
        else:
            return None

/tmp/ipykernel_706/1720371371.py:11: UserWarning: libcamera module not found. Use openCV instead.
  warnings.warn("libcamera module not found. Use openCV instead.")


In [3]:
cam = Camera()

In [4]:
from collections import deque
from time import time

display_handle = ipython.display(None, display_id=True)
captured_frame_display_handle = ipython.display(None, display_id=True)

start_btn = widgets.Button(description=" Start Camera", button_style="danger", icon="play")
capture_btn = widgets.Button(description=" Capture", disabled=True)
save_btn = widgets.Button(description="Save", disabled=True)

is_started = False
queue = deque()
frame = None
captured_frame = None

def start_cam():
    global is_started
    global frame
    try:
        cam.start()
        while is_started:
            frame = cam.get_frame()
            if frame is not None:
                _, img = cv2.imencode('.jpeg', frame)
                display_handle.update(ipython.Image(data=img.tobytes()))
    except KeyboardInterrupt:
        pass
    finally:
        cam.stop()

def on_capture(b):
    # get frame into captured_frame and display
    global captured_frame
    
    captured_frame = frame
    if captured_frame is not None:
        save_btn.disabled = False
        _, img = cv2.imencode('.jpeg', captured_frame)
        captured_frame_display_handle.update(ipython.Image(data=img.tobytes()))

def on_save(b):
    # save captured_frame into a file w/ the timestamp as the name
    global captured_frame
    
    if captured_frame is not None:
        cv2.imwrite(f"{int(time())}.jpg", captured_frame)
        b.disabled = True
        captured_frame = None
        captured_frame_display_handle.update(None)

def on_start(b):
    global is_started
    
    if is_started:
        is_started = False
        capture_btn.disabled = True
        b.description = " Start Camera"
        b.icon = "play"
        if len(queue) > 0:
            thread = queue.popleft()
            thread.join()
    else:
        is_started = True
        capture_btn.disabled = False
        b.description = " Stop Camera"
        b.icon = "stop"
        
        thread = threading.Thread(target=start_cam)
        thread.start()
        queue.append(thread)

start_btn.on_click(on_start)
capture_btn.on_click(on_capture)
save_btn.on_click(on_save)
display(start_btn, capture_btn, save_btn)

None

None

Button(button_style='danger', description=' Start Camera', icon='play', style=ButtonStyle())

Button(description=' Capture', disabled=True, style=ButtonStyle())

Button(description='Save', disabled=True, style=ButtonStyle())

[ WARN:0@5.266] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@5.267] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
